In [3]:
import requests
from bs4 import BeautifulSoup

baseUrl = 'https://www.wine.com'

page = requests.get(f'{baseUrl}/list/wine/red-wine/7155-124?showOutOfStock=true')
cont = 0
while cont < 1:
    soup = BeautifulSoup(page.text, 'html.parser')

    wineList = soup.find('ul', class_='listGridLayout_list')
    wineElements = wineList.find_all('li', class_='listGridLayout_listItem')

    # iterate <li> items
    for wineElem in wineElements[:3]:
        # Try <5 times to load the product page
        trials = 0
        while trials < 5:
            try:
                # is out of stock?
                outOfStock = wineElem.find('div', class_='productUnavailable')

                wineInfo = { 'color': 'red' }   # Iterate over red, white, ...

                # ----- Getting info from main page -----
                # Name and vintage  
                name = wineElem.find('span', itemprop='name').text
                vintage = name[-4:]
                wineInfo['name'] = name
                wineInfo['vintage'] = int(vintage)

                # Variety
                variety = wineElem.find('span', class_='listGridItemOrigin_varietal').text
                wineInfo['variety'] = variety

                # Origin
                region = wineElem.find('span', class_='listGridItemOrigin_text').text
                wineInfo['region'] = region

                # Price
                price = wineElem.find('meta', itemprop='price')['content']
                wineInfo['price'] = float(price)

                # Ratings
                ulRatings = wineElem.find('ul', class_='wineRatings_list')
                if ulRatings:
                    ratings = []
                    for ratingElem in ulRatings.find_all('li'):
                        ratingName = ratingElem['title'][:-16].strip()
                        ratingValue = ratingElem['title'][-15:-13].strip()
                        
                        rating = {'name': ratingName, 'rating': int(ratingValue)}

                        ratings.append(rating)

                    wineInfo['ratings'] = ratings

                # print(wineInfo['region'])
                # break

                # ----- Getting info from wine page -----
                
                # Get wine detail URL
                a = wineElem.find('a', class_='event_productClick', href=True)
                wineUrl = f"{baseUrl}{a['href']}"

                # Open wine page
                winePage = requests.get(wineUrl)
                wineSoup = BeautifulSoup(winePage.text, 'html.parser')       

                if outOfStock: # If out of stock, page is different
                    # Size
                    size = wineSoup.find('span', class_='prodAlcoholVolume_text').text
                    wineInfo['size'] = size

                    # Images
                    divImage = wineSoup.find('div', class_='pipThumbs')
                    images = []
                    for image in divImage.find_all('img'):
                        imageUrl = image['src'].split('/')[-1]
                        images.append('https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/' + imageUrl)
                else:
                    # Getting info from product details table
                    productDetails = wineSoup.find('section', class_='pipProdDetails')
                    titles = productDetails.find_all('div', class_='pipProdDetails_title')
                    values = productDetails.find_all('div', class_='pipProdDetails_name')
                    for i, title in enumerate(titles):
                        if title.text.strip() == 'Size':    # Size
                            wineInfo['size'] = values[i].text[:-2]
                        elif title.text.strip() == 'Producer':  # Producer
                            wineInfo['producer'] = values[i].text
                    
                    # Images
                    divImage = wineSoup.find('div', class_='pipProdThumbs')
                    images = []
                    for image in divImage.find_all('img'):
                        imageUrl = image['src'].split('/')[-1]
                        images.append('https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/' + imageUrl)
                
                wineInfo['image'] = images

                # Print wine
                print(wineUrl)
                for key, value in wineInfo.items():
                    print(f"{key}: {value}")
                print()
                
                break
            except Exception as err:
                trials += 1
                continue
        else:
            print(f"FAIL")

    nextPageUrl = soup.find('a', class_='listPageNextUrl')['href']
    page = requests.get(nextPageUrl)

    cont += 1

https://www.wine.com/product/antinori-badia-a-passignano-chianti-classico-gran-selezione-2018/806034
color: red
name: Antinori Badia a Passignano Chianti Classico Gran Selezione 2018
vintage: 2018
variety: Sangiovese
region: Chianti Classico, Chianti, Tuscany, Italy
price: 47.98
ratings: [{'name': 'James Suckling', 'rating': 95}, {'name': 'Wine Spectator', 'rating': 94}, {'name': 'Wine & Spirits', 'rating': 92}, {'name': 'Wine Enthusiast', 'rating': 90}]
producer: Antinori
size: 750
image: ['https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/tict3jscz7n7xdtvreyw.jpg', 'https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/d7lpaxzi8mdvly1e1hfq.jpg']

https://www.wine.com/product/le-cellier-des-princes-cotes-du-rhone-la-couronne-du-prince-2018/1079083
color: red
name: Le Cellier des Princes Cotes du Rhone La Couronne du Prince 2018
vintage: 2018
variety: Rhone Red Blends
region: Cotes du Rhone, Rhone, France
price: 14.98
ratings: [

In [8]:
import mysql.connector

# ----- MySQL Connection -----
conn = mysql.connector.connect(host='localhost',
                               database='thewinegame',
                               user='user',
                               password='password',
                               port=6033)
cursor = conn.cursor(buffered=True)

store_id = 1
currency = 'dolar'

wineInfo = { 
    'color': 'Red',
    'name': 'Antica Mountain Select Cabernet Sauvignon 2018',
    'vintage': 2018,
    'variety': ['Cabernet Sauvignon'],
    'region': 'Atlas Peak, Napa Valley, California',
    'price': 59.99,
    'ratings': [{'name': 'James Suckling', 'rating': 94}, {'name': "Robert Parkers Wine Advocate", 'rating': 93}, {'name': 'Jeb Dunnuck', 'rating': 92}, {'name': 'Wine Spectator', 'rating': 92}],
    'producer': 'Antica',
    'size': 750,
    'image': ['https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/vaqmy9xth7r9l69hsxdj.jpg', 'https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/ykuvjgp7ax3829j6alip.jpg']
}

try:
    # ----- Isolated Attributes -----

    # Color
    cursor.execute(f"INSERT INTO color (name) \
                    SELECT * FROM (SELECT '{wineInfo['color']}' AS name) AS tmp \
                    WHERE NOT EXISTS (SELECT id FROM color WHERE name='{wineInfo['color']}')")

    # Producer
    if 'producer' in wineInfo.keys():
        cursor.execute(f"INSERT INTO producer (name) \
                        SELECT * FROM (SELECT '{wineInfo['producer']}' AS name) AS tmp \
                        WHERE NOT EXISTS (SELECT id FROM producer WHERE name='{wineInfo['producer']}')")

    # Variety
    for variety in wineInfo['variety']:
        cursor.execute(f"INSERT INTO variety (name) \
                        SELECT * FROM (SELECT '{variety}' AS name) AS tmp \
                        WHERE NOT EXISTS (SELECT id FROM variety WHERE name='{variety}')")

    # Rating type
    if 'ratings' in wineInfo.keys():
        for rating in wineInfo['ratings']:
            cursor.execute(f"INSERT INTO rating_type (name) \
                            SELECT * FROM (SELECT '{rating['name']}' AS name) AS tmp \
                            WHERE NOT EXISTS (SELECT id FROM rating_type WHERE name='{rating['name']}')")

    # Region
    regions = wineInfo['region'].split(',')[::-1]
    for i, region in enumerate(regions):
        region = region.strip()

        if i == 0:
            cursor.execute(f"INSERT INTO region (name, region_id) \
                            SELECT * FROM (SELECT '{region}' AS name, NULL AS region_id) AS tmp \
                            WHERE NOT EXISTS (SELECT id FROM region WHERE name='{region}')")
        else:
            cursor.execute(f"INSERT INTO region (name, region_id) \
                            SELECT * FROM (SELECT '{region}' AS name, (SELECT id FROM region WHERE name='{regions[i-1]}') AS region_id) AS tmp \
                            WHERE NOT EXISTS (SELECT id FROM region WHERE name='{region}')")

    # ----- Wine -----
    if 'producer' in wineInfo.keys():
        cursor.execute(f"INSERT INTO wine (color_id, region_id, vintage, size, producer_id, name) \
                        SELECT * FROM (SELECT \
                                            (SELECT id FROM color WHERE name='{wineInfo['color']}') AS color_id, \
                                            (SELECT id FROM region WHERE name='{regions[-1]}') AS region_id, \
                                            {wineInfo['vintage']} AS vintage, \
                                            {wineInfo['size']} AS size, \
                                            (SELECT id FROM producer WHERE name='{wineInfo['producer']}') AS producer_id, \
                                            '{wineInfo['name']}' AS name \
                                    ) AS tmp \
                        WHERE NOT EXISTS (SELECT id FROM wine WHERE name='{wineInfo['name']}' AND vintage={wineInfo['vintage']})")
    else:
        cursor.execute(f"INSERT INTO wine (color_id, region_id, vintage, size, name) \
                        SELECT * FROM (SELECT \
                                            (SELECT id FROM color WHERE name='{wineInfo['color']}') AS color_id, \
                                            (SELECT id FROM region WHERE name='{regions[-1]}') AS region_id, \
                                            {wineInfo['vintage']} AS vintage, \
                                            {wineInfo['size']} AS size, \
                                            '{wineInfo['name']}' AS name \
                                    ) AS tmp \
                        WHERE NOT EXISTS (SELECT id FROM wine WHERE name='{wineInfo['name']}' AND vintage={wineInfo['vintage']})")

    # ----- Dependent Attributes -----

    sqlGetWineId = f"(SELECT id FROM wine WHERE name='{wineInfo['name']}' AND vintage={wineInfo['vintage']})"

    # Wine Variety
    for variety in wineInfo['variety']:
        sqlGetVarietyId = f"(SELECT id FROM variety WHERE name='{variety}')"
        cursor.execute(f"INSERT INTO wine_variety \
                        SELECT * FROM (SELECT {sqlGetWineId}, {sqlGetVarietyId}) AS tmp \
                        WHERE NOT EXISTS (SELECT wine_id FROM wine_variety WHERE wine_id={sqlGetWineId} AND variety_id={sqlGetVarietyId})")

    # Images
    for image in wineInfo['image']:
        cursor.execute(f"INSERT INTO image (link, wine_id) \
                        SELECT * FROM (SELECT '{image}', {sqlGetWineId}) AS tmp \
                        WHERE NOT EXISTS (SELECT id FROM image WHERE link='{image}' AND wine_id={sqlGetWineId})")

    # Ratings
    if 'ratings' in wineInfo.keys():
        for rating in wineInfo['ratings']:
            sqlGetRatingId = f"(SELECT id FROM rating_type WHERE name='{rating['name']}')"

            cursor.execute(f"INSERT INTO rating (wine_id, rating_type_id, value) \
                            SELECT * FROM (SELECT {sqlGetWineId}, {sqlGetRatingId}, {rating['rating']}) AS tmp \
                            WHERE NOT EXISTS (SELECT wine_id FROM rating WHERE wine_id={sqlGetWineId} AND rating_type_id={sqlGetRatingId})")

    # Price
    cursor.execute(f"INSERT INTO price (wine_id, store_id, value, currency) \
                    SELECT * FROM (SELECT {sqlGetWineId}, {store_id}, {wineInfo['price']}, '{currency}') AS tmp \
                    WHERE NOT EXISTS (SELECT id FROM price WHERE wine_id={sqlGetWineId} AND store_id={store_id})")

    conn.commit()
except Exception as err:
    print(err)
    conn.close()